In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import numpy as np
import os
import time
import random
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import pickle
#from tensorboardX import SummaryWriter
from tqdm import tqdm
import matplotlib.pyplot as plt
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

import pickle

seed =1
torch.manual_seed(seed)


from torchvision import transforms, utils
import torchvision.models as models
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

from multi_center_tools.multi_tools import axial_index_sampling,open_pickle, load_TCGA_arrays,select_df
from multi_center_tools.IDH_tools import TCGA_Dataset_With_sampling_from_index, TCGAToTensor
#from multi_center_tools.Cycle_tool import get_converted_arrys
import copy
import copy
from torchmetrics import F1Score
from sklearn.model_selection import KFold
f1ig0 = F1Score(num_classes=2, average='macro', ignore_index=0)
f1ig1 = F1Score(num_classes=2, average='macro', ignore_index=1)
from multi_center_tools.ViT_tools import get_ViT_series,get_center_indexes

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
csv_loc = '/home/satoc/HvsAI/label_data/TCGA_training_nondu.csv'
img_loc = '/home/satoc/arrays/kino_scaling/FR_BraTS2019_array'
train_list = open_pickle(
    '/home/satoc/HvsAI/label_data/TCGA_list1_train.pickle')

val_list = open_pickle(
    '/home/satoc/HvsAI/label_data/TCGA_list1_test.pickle')

df =pd.read_csv(csv_loc)
train_val_pd = select_df(df,'BraTS_2019_subject_ID',train_list )
val_pd = select_df(df,'BraTS_2019_subject_ID',val_list )

dict_path = '/home/satoc/HvsAI/data/official_models/resnet34-333f7ec4.pth'



Z_score = False
SZ = True
Double = False
inst = None

Batch_size = 32
weights = torch.tensor([1.0, 1.0]).to(device)
epoch = 100
lr = 5e-3

fold = 0
max_lr=lr
#total_steps=(1 + len(train_pd )//Batch_size )  * epoch
total_steps=int(1 + (len(train_val_pd)*0.8)//Batch_size )  * epoch
print(total_steps)

save_name = 'TCGA10fold_width3_fold' + str(fold)
print(save_name)

# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "./saved_models/" + save_name
if not os.path.exists(CHECKPOINT_PATH):
    os.makedirs(CHECKPOINT_PATH)

300
TCGA10fold_width3_fold0


In [3]:
kf = KFold(n_splits=10, shuffle=True, random_state=0)

In [4]:
train_indexs=[]
val_indexs = []
for train_index, val_index in kf.split(train_val_pd):
    print(train_index)
    print(val_index)
    train_indexs.append(train_index)
    val_indexs.append(val_index)

[  0   1   3   4   5   6   8   9  11  12  13  14  15  16  17  18  19  20
  21  23  24  25  27  28  29  30  31  32  34  35  36  37  38  39  40  41
  42  43  44  45  46  47  49  50  51  52  53  54  55  56  57  58  59  60
  61  62  63  64  65  67  68  69  70  71  72  73  74  75  76  77  79  80
  81  82  83  84  85  86  87  88  89  92  93  94  95  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111]
[ 2  7 10 22 26 33 48 66 78 90 91 96]
[  0   1   2   3   4   5   6   7   9  10  11  12  14  15  17  18  19  20
  21  22  23  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39
  40  41  42  43  44  46  47  48  49  50  51  52  53  54  55  56  57  58
  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  76  77
  78  79  80  81  83  84  85  86  87  88  90  91  94  95  96  97  98  99
 101 102 103 104 105 107 108 109 110 111]
[  8  13  16  24  45  75  82  89  92  93 100 106]
[  0   1   2   4   5   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25 

In [5]:
train_pd = train_val_pd.iloc[train_indexs[fold]]
val_pd = train_val_pd.iloc[val_indexs[fold]]

#delete level 0
train_pd = train_pd.iloc[:, 2:]
val_pd = val_pd.iloc[:, 2:]

train_pd = train_pd.reset_index()
val_pd = val_pd.reset_index()

In [6]:
train_pd

,index,BraTS_2019_subject_ID,training_or_validation,gender,race,age_at_initial_pathologic_diagnosis,karnofsky_performance_score,histological_type,IDH1,IDH2,IDH3B,CDKN3,TERT,IDH1_2
0,0,BraTS19_TCIA01_429_1,Training,MALE,WHITE,54,100,Glioblastoma,0,0,0,0,0,0
1,1,BraTS19_TCIA01_401_1,Training,MALE,WHITE,78,80,Glioblastoma,0,0,0,0,0,0
2,4,BraTS19_TCIA02_198_1,Training,MALE,WHITE,54,80,Glioblastoma,0,0,0,0,0,0
3,5,BraTS19_TCIA02_377_1,Training,FEMALE,WHITE,63,[Not Available],Glioblastoma,0,0,0,0,0,0
4,6,BraTS19_TCIA02_314_1,Training,MALE,WHITE,40,60,Glioblastoma,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,115,BraTS19_TCIA13_645_1,Training,MALE,WHITE,31,[Not Available],Oligoastrocytoma,1,0,0,0,0,1
96,116,BraTS19_TCIA13_630_1,Training,FEMALE,WHITE,44,80,Astrocytoma,1,0,0,0,0,1
97,117,BraTS19_TCIA13_621_1,Training,FEMALE,WHITE,40,[Not Available],Oligoastrocytoma,1,0,0,0,0,1
98,119,BraTS19_TCIA13_642_1,Training,MALE,WHITE,36,[Not Available],Oligoastrocytoma,1,0,0,0,0,1


In [7]:
train_pd['tumor_index'] = get_center_indexes(train_pd, img_loc)
c_train_pd = pd.DataFrame(get_ViT_series(train_pd, _width=1))
c_train_pd.reset_index()

val_pd['tumor_index'] = get_center_indexes(val_pd, img_loc)
c_val_pd = pd.DataFrame(get_ViT_series(val_pd, _width=1))
c_val_pd.reset_index()

,level_0,index,BraTS_2019_subject_ID,training_or_validation,gender,race,age_at_initial_pathologic_diagnosis,karnofsky_performance_score,histological_type,IDH1,IDH2,IDH3B,CDKN3,TERT,IDH1_2,tumor_index
0,0,2,BraTS19_TCIA02_491_1,Training,FEMALE,WHITE,81,[Not Available],Glioblastoma,0,0,0,0,0,0,55
1,0,2,BraTS19_TCIA02_491_1,Training,FEMALE,WHITE,81,[Not Available],Glioblastoma,0,0,0,0,0,0,56
2,0,2,BraTS19_TCIA02_491_1,Training,FEMALE,WHITE,81,[Not Available],Glioblastoma,0,0,0,0,0,0,57
3,1,8,BraTS19_TCIA02_430_1,Training,FEMALE,WHITE,53,[Not Available],Glioblastoma,0,0,0,0,0,0,90
4,1,8,BraTS19_TCIA02_430_1,Training,FEMALE,WHITE,53,[Not Available],Glioblastoma,0,0,0,0,0,0,91
5,1,8,BraTS19_TCIA02_430_1,Training,FEMALE,WHITE,53,[Not Available],Glioblastoma,0,0,0,0,0,0,92
6,2,11,BraTS19_TCIA02_374_1,Training,MALE,WHITE,63,80,Glioblastoma,0,0,0,0,0,0,83
7,2,11,BraTS19_TCIA02_374_1,Training,MALE,WHITE,63,80,Glioblastoma,0,0,0,0,0,0,84
8,2,11,BraTS19_TCIA02_374_1,Training,MALE,WHITE,63,80,Glioblastoma,0,0,0,0,0,0,85
9,3,24,BraTS19_TCIA02_274_1,Training,FEMALE,WHITE,54,80,Glioblastoma,0,0,0,0,0,0,83


In [8]:
s_train = TCGA_Dataset_With_sampling_from_index(c_train_pd,
                                                img_loc,axial_index_sampling,
                                                z_score=Z_score,sz=SZ, double=Double,inst=inst,
                                                transform = TCGAToTensor())
s_val = TCGA_Dataset_With_sampling_from_index(c_val_pd,
                                              img_loc,axial_index_sampling,inst=inst,
                                              z_score=Z_score,sz=SZ, double=Double,
                                              transform = TCGAToTensor())

train_loader = torch.utils.data.DataLoader(s_train, batch_size=Batch_size, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(s_val, batch_size=Batch_size, shuffle = True, num_workers=0)

In [9]:
print(len(s_train))
print(len(s_val))

300
36


In [10]:
s_train[0][0].shape

torch.Size([12, 176, 192])

# Moodel

In [11]:
offcial34 = models.resnet34()
offcial34.load_state_dict(torch.load(dict_path))

<All keys matched successfully>

In [12]:
offcial34.conv1 = nn.Conv2d(12, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
offcial34.fc = nn.Linear(in_features=512, out_features=2, bias=True)
net = offcial34

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
act_fn_by_name = {
    "tann":nn.Tanh,
    "relu":nn.ReLU,
    "leakyrelu":nn.LeakyReLU,
    "gelu":nn.GELU
}

print(pl.__version__)

1.6.2


In [14]:
# Setting the seed
pl.seed_everything(42)

Global seed set to 42


42

In [15]:
class JCModule(pl.LightningModule):
    
    def __init__(self, model, optimizer_name, optimizer_hparams):
        """
        Inputs:
              optimizer_name - Name of the optimzer to use. Currently supported: Adam, SGD
              optimizer_hparams - Hyperparameters for the optimizer, as dictiornay. This includes learning rate weight decay, etc.
        """
        super().__init__()
        #Exports the hyperparameters to YAML file, and create "self.hparams" namespace
        self.save_hyperparameters()
        # Create model
        self.model = model
        # Create loss module
        self.loss_module = nn.CrossEntropyLoss()
        #Example input for visualizing the graph in Tensorboard
        self.example_input_array = torch.zeros((1,12,176,192), dtype=torch.float32)
        
    def forward(self, imgs):
        # Forward function that is run when visualizing the graph
        return self.model(imgs)
    
    def configure_optimizers(self):
        # We will support Adam or SGD as optimizers.
        if self.hparams.optimizer_name == "Adam":
            #AdamW is Adam with a correct implementation of weight decay
            optimizer = optim.AdamW(
            self.parameters(), **self.hparams.optimizer_hparams)
        elif self.hparams.optimizer_name == "SGD":
            optimizer = optim.SGD(self.parameters(),**self.hparams.optimizer_hparams)
        else:
            assert False, f"Unknown optimizer: \"{self.hparams.optimizer_name}\""
        # We will redudce the learning rate by 0.1 after 100 and 150 epochs
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr,total_steps=total_steps)
        return [optimizer], [scheduler]
        
    def training_step(self, batch, batch_idx):
        # "batch" is the output of the training data loader.
        imgs, labels = batch
        labels = labels[3]
        preds = self.model(imgs)
        loss = self.loss_module(preds, labels)
        acc = (preds.argmax(dim =-1) == labels).float().mean()
        f1score = f1ig0 (preds, labels)
        f1scoreR = f1ig1 (preds, labels)
        #Logs the accuracy per epoch to tensorboard (weighted average over batches)
        self.log('train_acc',acc,on_step=False, on_epoch = True)
        self.log('train_loss', loss)
        self.log('train_F1', f1score)
        self.log('train_F1R', f1scoreR)
        return loss # Return tensor to call ".backward" on
    
    def validation_step(self, batch, batch_idx):
        imgs, labels = batch
        labels = labels[3]
        preds = self.model(imgs).argmax(dim =-1)
        acc = (labels == preds).float().mean()
        f1score = f1ig0 (preds, labels)
        f1scoreR = f1ig1 (preds, labels)
        # By default logs it per epoch (weighted average over batches)
        self.log('val_acc', acc)
        self.log('val_F1', f1score)
        self.log('val_F1R', f1scoreR)
        
    def test_step(self, batch, batch_idx):
        imgs, labels = batch
        labels = labels[3]
        preds = self.model(imgs).argmax(dim =-1)
        acc = (labels == preds).float().mean()
        f1score = f1ig0 (preds, labels)
        f1scoreR = f1ig1 (preds, labels)
        # By the default logs it per epoch(weight average over batches), and return it afterwards
        self.log('test_acc', acc)
        self.log('test_F1', f1score)
        self.log('test_F1R', f1scoreR)
        
    def predict_step(self, batch, batch_idx):
        imgs, labels = batch
        preds = self.model(imgs).argmax(dim =-1)
        return preds

In [16]:
def train_model(models,save_name, **kwargs):
    """
    Inputs:
        model_name - Name of the model you want to run. It used to look up the class in "model_dict"
        save_name(optitional) - If specified, this name will be used for creating checkpoint and logging directory.
    """
    
    # Create a Pytorch Lightning trainer with the generation callback
    trainer = pl.Trainer(default_root_dir = os.path.join(CHECKPOINT_PATH, save_name),
                        gpus=1 if str(device)=='cuda:0' else 0,
                        max_epochs =100,
                        callbacks=[ModelCheckpoint(save_on_train_epoch_end='True',mode="max", monitor="val_acc"),
                                  LearningRateMonitor("epoch")],
                        progress_bar_refresh_rate=1)
    trainer.logger._log_graph = True         #If True, we plot the computation graph in tensorboard
    trainer.logger._default_hp_metric = None
    
    pl.seed_everything(42) # To be reproducable
    model = JCModule(models, **kwargs)
    trainer.fit(model, train_loader, val_loader)
    model = JCModule.load_from_checkpoint(trainer.checkpoint_callback.best_model_path) # Load best checkpoint after training
        
    # Test best model on validation and test set
    val_result = trainer.test(model, dataloaders=val_loader, verbose=False)
    train_result = trainer.test(model, dataloaders=train_loader, verbose=False)
    result = {"train_acc":train_result[0]["test_acc"], "train_F1":train_result[0]["test_F1"],
              "train_F1R":train_result[0]["test_F1R"],
              "val_acc":val_result[0]["test_acc"],"val_F1":val_result[0]["test_F1"],
             "val_F1R":val_result[0]["test_F1R"]}
    
    return model, result

In [17]:
resnet_model, resnet_results = train_model(net,save_name='ResNet_TCGA_CycleLR', 
                                           optimizer_name="SGD",
                                           optimizer_hparams={"lr": lr,
                                                              "momentum": 0.9,
                                                              "weight_decay": 1e-4})

/uge_mnt/home/satoc/.raiden/nvcr-pytorch-2203/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=1)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
Missing logger folder: saved_models/TCGA10fold_width3_fold0/ResNet_TCGA_CycleLR/lightning_logs

  | Name        | Type             | Params | In sizes          | Out sizes
---------------------------------------------------------------------------------
0 | model       | ResNet           | 21.3 M | [1, 12, 176, 192]

Epoch 0:  83%|████████▎ | 10/12 [05:20<01:04, 32.09s/it, loss=0.729, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 1:  83%|████████▎ | 10/12 [11:24<02:16, 68.46s/it, loss=0.641, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 2:  83%|████████▎ | 10/12 [17:23<03:28, 104.35s/it, loss=0.504, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 3:  83%|████████▎ | 10/12 [23:10<04:38, 139.10s/it, loss=0.388, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 4:  83%|████████▎ | 10/12 [29:12<05:50, 175.26s/it, loss=0.27, v_num=0] 
Validation: 0it [00:00, ?it/s]
Epoch 5:  83%|████████▎ | 10/12 [35:06<07:01, 210.66s/it, loss=0.176, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 6:  83%|████████▎ | 10/12 [40:57<08:11, 245.77s/it, loss=0.109, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 7:  83%|████████▎ | 10/12 [46:42<09:20, 280.23s/it, loss=0.0664, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 8:  83%|████████▎ | 10/12 [52:27<10:29, 314.77s/it, loss=0.049, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 9:  8

In [18]:
print("ResNet Results", resnet_results)

ResNet Results {'train_acc': 1.0, 'train_F1': 1.0, 'train_F1R': 1.0, 'val_acc': 1.0, 'val_F1': 1.0, 'val_F1R': 1.0}
